In [1]:
from sequential_tasks import TemporalOrderExp6aSequence
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import SimpleRNN, Dense


In [2]:
def exp6a_experiment(settings):
    train_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
        settings['difficulty'],
        settings['batch_size'])

    model = Sequential([
        SimpleRNN(
            units=settings['h_units'],
            input_shape=(train_data_gen.length_range[1],
                         train_data_gen.n_symbols)),
        Dense(units=train_data_gen.n_classes, activation='softmax')
    ])

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        generator=train_data_gen,
        epochs=settings['max_epochs'],
        verbose=2)

    # testing
    test_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
        settings['difficulty'],
        settings['batch_size'])

    eval_metrics = model.evaluate_generator(test_data_gen)
    test_accuracy = eval_metrics[1]
    
    return test_accuracy


In [3]:
# experiments settings
params = {
    "difficulty": TemporalOrderExp6aSequence.DifficultyLevel.EASY,
    "batch_size": 32,
    "h_units": 4,
    "max_epochs": 30
}

acc = exp6a_experiment(params)
print('acc = {:.2f}%.'.format(acc))


Epoch 1/30
 - 0s - loss: 1.3959 - acc: 0.2591
Epoch 2/30
 - 0s - loss: 1.3335 - acc: 0.3347
Epoch 3/30
 - 0s - loss: 1.3186 - acc: 0.3498
Epoch 4/30
 - 0s - loss: 1.2981 - acc: 0.3659
Epoch 5/30
 - 0s - loss: 1.2400 - acc: 0.4355
Epoch 6/30
 - 0s - loss: 1.2026 - acc: 0.4899
Epoch 7/30
 - 0s - loss: 1.1653 - acc: 0.5101
Epoch 8/30
 - 0s - loss: 1.1257 - acc: 0.5232
Epoch 9/30
 - 0s - loss: 1.0727 - acc: 0.5817
Epoch 10/30
 - 0s - loss: 1.0358 - acc: 0.6431
Epoch 11/30
 - 0s - loss: 0.9817 - acc: 0.6643
Epoch 12/30
 - 0s - loss: 0.9390 - acc: 0.6825
Epoch 13/30
 - 0s - loss: 0.8946 - acc: 0.7016
Epoch 14/30
 - 0s - loss: 0.8541 - acc: 0.7097
Epoch 15/30
 - 0s - loss: 0.8061 - acc: 0.7681
Epoch 16/30
 - 0s - loss: 0.7683 - acc: 0.7621
Epoch 17/30
 - 0s - loss: 0.7158 - acc: 0.7944
Epoch 18/30
 - 0s - loss: 0.6629 - acc: 0.8730
Epoch 19/30
 - 0s - loss: 0.6269 - acc: 0.9153
Epoch 20/30
 - 0s - loss: 0.5782 - acc: 0.9345
Epoch 21/30
 - 0s - loss: 0.5307 - acc: 0.9556
Epoch 22/30
 - 0s - lo